In [7]:
import os
import xmltodict
import pandas as pd
import numpy as np
from html.parser import HTMLParser  # Python 3
from IPython.display import clear_output

dirname = os.path.join('data', 'Bayzick')

messages = pd.DataFrame()
consensus = pd.DataFrame()
print("Running....")
num_files = len(os.listdir(dirname))
num_errors = 0
for filename in os.listdir(dirname):
    if filename in [os.listdir(dirname)[7], os.listdir(dirname)[8], os.listdir(dirname)[10], os.listdir(dirname)[15]]:
        continue
    if '.xml' in filename:
        try:
            file = dirname + '/' + filename
            messages_df = pd.DataFrame(pd.DataFrame(xmltodict.parse(open(file, "rb")))['posts'][0])
            messages_df['origin'] = filename.strip('.xml')
            messages = pd.concat([messages, messages_df])
        except:
            num_errors += 1
            print("error in {}".format(file))
    elif '.xlsx' in filename:
        file = dirname + '/' + filename
        # We need to ignore the first 2 lines of these .xlsx files
        consensus = pd.concat([consensus, pd.DataFrame(pd.read_excel(open(file, 'rb'),converters={'File Name':str}, skiprows=[0,1]))])

print(messages[['body', 'origin']])
print(num_errors, num_files)
clear_output()


In [8]:
consensus = consensus.rename(columns={"File Name": "origin"})
print(consensus.columns)
messages = messages.merge(consensus)

Index(['origin', 'Is Cyberbullying Present?', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')


In [9]:
bayzick_lbl = list()

def concat_msg(group_df, bayzick_lbl=bayzick_lbl):
#     global bayzick_lbl
    concat_msg = group_df['body'].astype('str').str.cat().lower()
    is_bulleying = 1 if group_df.iloc[0]['Is Cyberbullying Present?'] == 'Y' else 0
    bayzick_lbl = bayzick_lbl.append([is_bulleying, concat_msg])

messages.groupby('origin').apply(concat_msg)
label_df = pd.DataFrame(bayzick_lbl, columns=['Label', 'Text']).iloc[4:]

In [13]:
label_df.to_csv('final_data/bayzick_label.csv', index=False)